# MBA em Ciência de Dados
## Técnicas Avançadas de Captura e Tratamento de Dados


### <span style="color:darkred">Módulo VII - Dados não estruturados: sinais e imagens</span>

### <span style="color:darkred">Exercícios</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

#### <span style="color:red">Recomenda-se fortemente que os exercícios sejam feitos sem consultar as respostas antecipadamente.</span>

---

In [1]:
# carregando as bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Exercício 3)

Carregue os dados do arquivo `sinais2.csv` utilizando

`signals = np.genfromtxt(arquivo, delimiter=',').astype(np.float32)`.

O array resultante possui um sinal por linha, i.e. `sinal[i]`

Utilizando os sinais carregados utilize a `np.fft.fft()` para obter a Transformada de Fourier dos sinais. Depois, considerando apenas frequências até 50, calcule quais são as 4 frequências de maior valor de magnitude (obtido pelo `np.abs()`). Aqui não queremos os valores da magnitude, mas a quais frequências (índices) elas se referem. Para complementar a análise, plote as magnitudes das transformadas até a frequência 50.

Analisando as frequências de maior magnitude temos as frequências que mais caracterizam o sinal. Considerando as 4 frequências computadas anteriormente, podemos dividir os sinais em categorias distintas. Nesse sentido, qual análise abaixo está correta?

(a) O sinal 4 possui frequências inferiores quando comparado com os demais, indicando que o sinal 4 é provavalmente  dependente sequencialmente, enquanto os demais são i.i.d.; assim podemos dividí-los em duas categorias: sinal 4 e sinais 0, 1, 2 e 3.<br>
(b) O sinal 3 possui frequências mais significativas 20 Hz ou superior, indicando que é um sinal com maior qualidade de aquisição, e assim podemos categorizar em: sinal 3, e sinais 0, 1, 2 e 4.<br>
(c) Todas as frequências estão abaixo de 50 Hz, sendo assim podemos dizer que os sinais são todos similares, sendo impossível dividí-los em categorias.<br>
(d) O sinal 3 possui frequências mais significativas 20 Hz ou superior, possuindo transições mais rápidas de valores do que os outros com frequências caracerísticas menores do que 12Hz; e assim podemos categorizar em: sinal 3, e sinais 0, 1, 2 e 4.<br>

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
import seaborn as sns
sns.set()
%load_ext autoreload
%autoreload 2

In [12]:
# Pode fazer com pandas tbm
# pd.read_csv("dados/sinais2.csv", header=None)
signals = np.genfromtxt('./dados/sinais2.csv', delimiter=',').astype(np.float32)
signals.shape

(5, 9000)

In [29]:
for i in range(len(signals)):
    Fi = np.abs(np.fft.fft(signals[i]))
    Fi_50 = Fi[:50]
    ind = np.argpartition(Fi_50, -4)[-4:]
    print(i, ind)

0 [10 11  8  6]
1 [10  9  8  7]
2 [7 5 9 8]
3 [28 39 21 20]
4 [3 8 2 6]


### Exercício 4)
 
Considerando os mesmos sinais carregados, compute as características: entropia da energia (com 10 blocos), taxa de cruzamentos por zero, entropia espectral (com 10 blocos), formando um vetor com 3 características para cada sinal.

Após isso, compute a matriz de distâncias entre os sinais considerando a distância L1, i.e., a soma dos valores absolutos das diferenças entre dois vetores $A$ e $B$:

$$\sum_i |A_i - B_i|$$

Da matriz, que indica a dissimilaridade entre pares de sinais, aplique uma soma na direção do eixo 0 (axis=0) e depois arredonde para inteiro `np.round(,0)`. Quais valores foram obtidos para cada sinal?

(a) Sinais 0, 1, 2 e 4, soma 2; Sinal 3, soma 6.<br>
(b) Sinais 0 e 4, soma 3; Sinais 1 e 2, soma 2; Sinal 3, soma 6.<br>
(c) Sinais 0, 1, e 2, soma 2; Sinal 3, soma 6; Sinal 4, soma 3.<br>
(d) Sinais 0, 1, e 2, soma 1; Sinal 3, soma 3; Sinal 4, soma 6.<br>

In [32]:
def entropia_energia(sinal, n_blocos=10):
    '''Entropia da energia do sinal'''
    # energia total 
    energia_sinal = np.sum(sinal ** 2)
    M = len(sinal)
    
    # calcula janelas dentro do sinal
    M_janelas = int(np.floor(M / n_blocos))
    # verifica se tamanho dos blocos 
    # é multiplo do tamanho do sinal
    if M != M_janelas * n_blocos:
        sinal = sinal[0:M_janelas * n_blocos]

    # monta matriz [M_janelas x n_blocos]
    janelas = sinal.reshape(M_janelas, n_blocos, order='F').copy()
    
    # Computa energias de cada janela (normalizada pela do sinal)
    e_janelas = np.sum(janelas ** 2, axis=0) / (energia_sinal + 0.0001)
    #print(e_janelas)

    # Computa entropia entre energias das janelas
    entropia = -np.sum(e_janelas * np.log2(e_janelas + 0.0001))
    return entropia

def taxa_cruzamentos_por_zero(sinal):
    '''Cruzamentos por zero em um intervalo de tempo '''
    M = len(sinal)
    cont_zero = np.sum(np.abs(np.diff(np.sign(sinal)))) / 2
    return np.float64(cont_zero) / np.float64(M - 1.0)

def entropia_espectral(sinal, n_blocos=10):
    """Computes the spectral entropy"""
    
    fft_abs = np.abs(np.fft.fft(sinal))
    
    entropia_esp = entropia_energia(fft_abs, n_blocos=n_blocos)

    return entropia_esp

In [40]:
funcs = [entropia_energia, taxa_cruzamentos_por_zero, entropia_espectral]
features = np.zeros((len(signals), len(funcs)))

for idx_signal, signal in enumerate(signals):
    for idx_func, func in enumerate(funcs):
        value = func(signal)
        features[idx_signal, idx_func] = value

# Pode ser interessante visualizar os dados com DataFrame para entender o que está acontecendo
pd.DataFrame(features, columns=["entropia_energia", "taxa_cruzamentos_por_zero", "entropia_espectral"])

In [47]:
dmat = np.zeros((len(signals), len(signals)))
for idx_signal1, feat1 in enumerate(features):
    for idx_signal2, feat2 in enumerate(features):
        dmat[idx_signal1, idx_signal2] = np.abs(feat1 - feat2).sum()
dmat

array([[0.        , 0.08820419, 0.16471197, 1.55867705, 0.56184067],
       [0.08820419, 0.        , 0.11717896, 1.55371289, 0.47363648],
       [0.16471197, 0.11717896, 0.        , 1.60484023, 0.42113137],
       [1.55867705, 1.55371289, 1.60484023, 0.        , 1.63633509],
       [0.56184067, 0.47363648, 0.42113137, 1.63633509, 0.        ]])

In [48]:
dmat.sum(axis=0).round(0)

array([2., 2., 2., 6., 3.])

### Exercício 5)

Carregue as seguintes imagens da base de dados flickr_map_training:

`
img1 = imageio.imread("dados/flickr_map_training/107.jpg")
img2 = imageio.imread("dados/flickr_map_training/101.jpg")
img3 = imageio.imread("dados/flickr_map_training/112.jpg")
img4 = imageio.imread("dados/flickr_map_training/303.jpg")
img5 = imageio.imread("dados/flickr_map_training/400.jpg")`

Implemente um descritor de cor que computa um histograma utilizando a composição dos canais RGB em um único canal utilizando a seguinte operação, sendo R, G e B as matrizes relativas a cada canal de cor:

$$I = R\cdot0.3 +G\cdot0.59 +B\cdot0.11$$

Permita definir o número de bins do histograma por meio da sua função e, antes de retornar, normalize o histograma dividindo pela soma.

Depois, calcule a distância entre img1 carregada e as outras imagens (2, 3, 4, 5) utilizando: 16 bins e 4 bins. Qual foram as duas imagens mais similares, da mais próxima para a mais distante, nos dois casos?

(a) 16 bins: img2, img4 ; 4 bins: img2, img3<br>
(a) 16 bins: img2, img3 ; 4 bins: img4, img3<br>
(b) 16 bins: img2, img3 ; 4 bins: img2, img4<br>
(d) 16 bins: img4, img2 ; 4 bins: img4, img3<br>

In [53]:
import imageio
img1 = imageio.imread("dados/flickr_map_training/107.jpg")
img2 = imageio.imread("dados/flickr_map_training/101.jpg")
img3 = imageio.imread("dados/flickr_map_training/112.jpg")
img4 = imageio.imread("dados/flickr_map_training/303.jpg")
img5 = imageio.imread("dados/flickr_map_training/400.jpg")

In [63]:
def histograma_global_intensity(img, n_colors):
    img_int = img[:,:,0].astype(float)*0.3 + img[:,:,1].astype(float)*0.59 + img[:,:,2].astype(float)*0.11
    hist,_ = np.histogram(img_int, bins=n_colors)
    # normaliza o vetor resultante pela soma dos valores
    hist = hist.astype("float")
    hist /= (hist.sum() + 0.0001)        
    return hist

In [81]:
# Mostrando que todas as imagens tem o mesmo min/max, que vai resultar
# numa binnização igual (com cortes iguais para cada bin)
for img in all_imgs:
    print(img.min(), img.max())

0 255
0 255
0 255
0 255
0 255


In [73]:
n_colors = 16

# Fazendo através de uma função para re-utilizar o código
def ex5_dist(n_colors):
    features = []
    all_imgs = [img1, img2, img3, img4, img5]
    for img in all_imgs:
        features.append(histograma_global_intensity(img, n_colors))

    dists = []
    for feat in features:
        dist = np.abs(features[0] - feat).sum()
        dists.append(dist)
    return dists

In [82]:
# sns.histplot(np.arange(10))
# sns.histplot(2 + np.arange(10))

In [76]:
ex5_dist(16)  # 2, 4 são as com menor distância

[0.0,
 0.22952025985165178,
 0.8369060998124469,
 0.7622912638468645,
 0.9559461472638263]

In [77]:
ex5_dist(4)  # 2, 3 são as com menor distância

[0.0,
 0.14886455508209526,
 0.5596221968952861,
 0.6472747798467804,
 0.7926671751897034]

### Exercício 6)

Vamos repetir o procedimento da questão anterior, agora utilizando o descritor de texturas LBP visto em aula. Utilizaremos uma função que também realiza uma normalização dos valores máximos das imagens, bem como permite definir o raio, número de pontos e quantidade de bins para esse descritor, conforme abaixo.

Calcule a distância L1 entre img1 carregada e as outras imagens utilizando o descritor LBP com os seguintes parâmetros:
* número de pontos = 14
* raio = 2
* bins = 16

Quais foram as três imagens mais similares, da mais próxima para a mais distante?

(a) img3, img2, img5<br>
(b) img2, img3, img4<br>
(c) img3, img5, img2<br>
(d) img5, img3, img2<br>

In [83]:
from skimage import feature

def lbp_features(img, points=8, radius=1, n_bins=10):
    # LBP opera em imagens de um só canal, aqui vamos converter 
    # RGB para escala de cinza usando o método Luminance
    img = np.array(img, dtype=np.float64, copy=False)
    if (len(img.shape) > 2):
        img = img[:,:,0]*0.3 + img[:,:,1]*0.59 + img[:,:,2]*0.11
    
    # normaliza a imagem para ter máximo = 255
    if (np.max(img) > 0):
        img = ((img/np.max(img))*255).astype(np.uint8)
    
    # aqui definimos o numero de pontos e o raio, padrao = 8, 1
    lbp = feature.local_binary_pattern(img.astype(np.uint8), points, radius, method="uniform")
    
    # lbp retorna um matriz com os códigos, então devemos extraír o histograma
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_bins+1), range=(0, n_bins))

    # normaliza o histograma
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    # return the histogram of Local Binary Patterns

    return hist

In [101]:
# reutilizando o código da função 5, so mudando a função
def ex6_dist():
    features = []
    all_imgs = [img1, img2, img3, img4, img5]
    for img in all_imgs:
        # Mudando a função
        features.append(lbp_features(img, points=14, radius=2, n_bins=16))

    dists = []
    for feat in features:
        dist = np.sum(np.abs(features[0] - feat))
        dists.append(dist)
    return dists

In [102]:
dists = ex6_dist()
pd.DataFrame(ex6_dist(), index=1 + np.arange(5))  # 3, 2, 5 é a ordem

,0
1,0.000000
2,0.155906
3,0.148553
4,0.457973
5,0.210260


In [95]:
# Se quisermos os índices da ordem
ind = np.argsort(dists)
ind

array([0, 2, 1, 4, 3])

In [166]:
# se quisermos as distâncias de acordo com os índices acima
dists = np.array(dists)
dists[ind]

array([0.        , 0.14855343, 0.15590619, 0.21026018, 0.45797315])

### Exercício 7)
 
No método Bag-of-Features quais das seguintes escolhas para o *framework* influenciam mais drasticamente a performance do método no caso de uso em imagens?

(a) O tamanho do dicionário, a quantidade de cores nas imagens, a quantidade de classes do problema<br>
(b) O tamanho do dicionário, o descritor base, o método utilizado para aprender o dicionário<br>
(c) O descritor base e o número de componentes principais utilizados<br>
(d) O tamanho do patch extraído da imagem, que deve ser compatível com a resolução das imagens<br>

In [ ]:
# ver discussão na gravação

### Exercício 8)

Execute o método Bag-of-Features estudado em aula, agora com os seguintes parâmetros:
* tamanho do patch = (13, 13)
* número de patches = 1000
* principais componentes = 10
* tamanho do dicionário = 50

Utilize a imagem de consulta `flower.jpg` e recupere as 12 imagens mais similares utilizando o modelo BoF aprendido. Quantas imagens foram recuperadas pertencendo à mesma categoria da consulta?

(a) 3<br>
(b) 0<br>
(c) 6<br>
(d) 9<br>

In [130]:
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from imageio import imread
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from joblib import Parallel, delayed


def get_patches(img_file, random_state, tam_patch=(11, 11), n_patches=250):
    '''Extração de subimagens a partir de uma imagem
       Parametros
           img_file: caminho para a imagem
           random_state: semente aleatoria
           tam_patches: tamanho de cada subimagem
           n_patches: numero maximo de subimagens a extrair
    '''

    img = imread(img_file)
    
    # Extrai subimagens
    patch = extract_patches_2d(img, 
                               patch_size=tam_patch,
                               max_patches=n_patches, 
                               random_state=random_state)
    
    # so da certo para RGB? -> Discutido durante a monitoria
    return patch.reshape((n_patches, 
                          np.prod(tam_patch) * len(img.shape)))
    # return patch.reshape((n_patches, -1))

In [131]:
# Parametros do BOF
tam_patch = (13, 13)
n_patches = 1000
path_imgs = './dados/flickr_map_training/'
random_state = 1
# pega lista de arquivos no caminho
l_imgs = listdir(path_imgs)

# total de imagens
n_imgs = len(l_imgs)

In [132]:
%%time
# Calculando quanto tempo demora no for na mão, dps vamos comparar com a versão paralela
for arq_img in l_imgs:
    get_patches(path_imgs+arq_img,  random_state, tam_patch, n_patches)

CPU times: user 206 ms, sys: 8.01 ms, total: 214 ms
Wall time: 214 ms


In [145]:
%%time
# Fazendo de maneira paralela; na minha máquina não tem grande vantagem

# Extrai patches de cada imagem, de forma paralela para cada imagem
# retorna uma lista do mesmo tamanho do número de imagens
patch_arr = Parallel(n_jobs=-1)(delayed(get_patches)(path_imgs+arq_img, 
                                                    random_state,
                                                    tam_patch,
                                                    n_patches)
                                for arq_img in l_imgs)

print('Patches extraídos para criação do dicionário de features')
print('Total de imagens = ', len(patch_arr))
print('Tamanho de cada array de patches = ', patch_arr[0].shape)

Patches extraídos para criação do dicionário de features
Total de imagens =  80
Tamanho de cada array de patches =  (1000, 507)
CPU times: user 142 ms, sys: 22.8 ms, total: 165 ms
Wall time: 183 ms


In [147]:
# cada patch vira uma linha
patch_arr2 = np.array(patch_arr, copy=True)
patch_arr2 = patch_arr2.reshape(patch_arr2.shape[0]*patch_arr2.shape[1], -1)

In [154]:
# Calculando PCA
pca = PCA(n_components=10)
pca.fit(patch_arr2)
patch_pca = pca.transform(patch_arr2)

In [156]:
patch_pca.shape

(80000, 10)

In [157]:
# Fazendo o clustering que vai definir meu dicionário
kmeans = KMeans(n_clusters=50,)
kmeans.fit(patch_pca)

KMeans(n_clusters=50)

In [167]:
# O que faltaria para terminar o exercício: colocar num for e fazer 1 distância L1;
# ver notebook com solução pro resto!
img_kmeans = kmeans.predict(patch_pca[0:1000])
hist_bof,_ = np.histogram(img_kmeans, bins=range(50+1), density=True)
hist_bof

array([0.023, 0.   , 0.   , 0.009, 0.   , 0.   , 0.   , 0.031, 0.   ,
       0.002, 0.003, 0.112, 0.095, 0.   , 0.   , 0.011, 0.007, 0.007,
       0.018, 0.037, 0.001, 0.01 , 0.006, 0.012, 0.   , 0.291, 0.   ,
       0.   , 0.107, 0.   , 0.002, 0.002, 0.011, 0.   , 0.   , 0.   ,
       0.024, 0.003, 0.092, 0.   , 0.008, 0.04 , 0.006, 0.   , 0.006,
       0.002, 0.022, 0.   , 0.   , 0.   ])